# <font color=green>Restaurant-Guru

> Este projeto tem como finalidade testar os conhecimentos que obtive completando a formação "Python para Data Science" da plataforma Alura. Aqui irei testar meus conhecimentos acerca de Web Scrapping e EDA (Exploratory Data Analysis).

<h4>Ideia Central:</h4> 

> A ideia do projeto é fazer um Web Scrapping do site https://restaurantguru.com.br/ e conseguir extrair informações valiosas por meio da análise de dados, gráficos e etc.

<h4>Sobre o site:</h4> 

> Restaurant Guru é um wiki+metasearch para restaurantes. Eles fornecem aos usuários informações completas sobre restaurantes como: avaliações dos visitantes, cardápios, fotos, etc. O principal objetivo do site é ajudar pessoas a escolherem o lugar certo para comer, seja em sua cidade ou durante sua viagem.

## <font color=blue>Importando Bibliotecas

> Neste passo iremos importar todas as bibliotecas que iremos utilizar nesse projeto

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import time

## <font color=blue>Web Scrapping

<img src='img/extrair.png'>

A imagem acima mostra os campos que iremos extrair do site. Vamos obter o **nome**, a **localização**, o **tipo de comida**, as **avaliações** e o **valor** de cada restaurante.

#### Configurações básicas - Obtenção de URL

In [2]:
url = 'https://restaurantguru.com.br/restaurant-Brazil-t1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

req = Request(url, headers = headers)
response = urlopen(req)
html = response.read()

#### Configurações básicas - Tratamento do HTML



> O tipo da variável *html* é **bytes**, executando o comando *html.decode('utf-8')* irei corrigir a acentuação de palavras e converter essa variável para **str**.

In [3]:
type(html)

bytes

In [4]:
html = html.decode('utf-8')

In [5]:
type(html)

str

>Elaboração de uma função para eliminar os caracteres de tabulação, quebra de linha e espaçamento entre TAGS. Após isso, foi aplicado a variável *html* para ser tratada.

In [6]:
def trata_html(input):
    return " ".join(html.split()).replace('> <', '><')

In [7]:
html = trata_html(html)

#### Criando um objeto BeautifulSoup

In [8]:
soup = BeautifulSoup(html, 'html.parser')

In [9]:
type(soup)

bs4.BeautifulSoup

#### Criando variávels para armazenar informações

In [10]:
cards = []
card = {}

### <font color=#3498DB>Obtendo os dados do primeiro CARD

In [11]:
anuncio = soup.find('div', {'class': 'restaurant_row show'})

**Obtendo o <font color=red>NOME</font> do estabelecimento:**

In [12]:
# Obtendo o título
nome = anuncio.find('div', {'class': 'title'}).get_text()
nome

'Figueira Rubayat / Restaurante, Churrascaria'

In [13]:
# Removendo o subtítulo
nome = nome.split(' /')[0]
nome

'Figueira Rubayat'

In [14]:
card['nome'] = nome

In [15]:
card

{'nome': 'Figueira Rubayat'}

**Obtendo o <font color=red>LOCALIZAÇÃO</font> do estabelecimento:**

In [16]:
local = anuncio.find('div', {'class': 'number'}).get_text()
local

'#1 de 153703 lugar para comer em São Paulo'

In [17]:
local = local.split()
local

['#1', 'de', '153703', 'lugar', 'para', 'comer', 'em', 'São', 'Paulo']

In [18]:
local = local[7:]
local

['São', 'Paulo']

In [19]:
local = [' '.join(local)]
local = str(local).strip("'[]'")
local

'São Paulo'

In [20]:
type(local)

str

In [21]:
card['local'] = local

In [22]:
card

{'nome': 'Figueira Rubayat', 'local': 'São Paulo'}

**Obtendo o <font color=red>TIPOS DE COMIDA</font> do estabelecimento:**

In [23]:
comida = anuncio.find('div', {'class': 'info cuisine'}).get_text()
comida

'Brasileira, Argentina, Grelhada, Churrascarias'

In [24]:
comida = comida.split(', ')
comida

['Brasileira', 'Argentina', 'Grelhada', 'Churrascarias']

In [25]:
card['comida'] = comida

In [26]:
card

{'nome': 'Figueira Rubayat',
 'local': 'São Paulo',
 'comida': ['Brasileira', 'Argentina', 'Grelhada', 'Churrascarias']}

**Obtendo o <font color=red>VALOR</font> do estabelecimento:**

In [27]:
valor = anuncio.find('span', {'class': 'cost'}).findAll('i')
valor

[<i>$</i>, <i>$</i>, <i>$</i>, <i>$</i>]

In [28]:
valor = len(valor)
valor

4

In [29]:
if valor == 4:
    card['valor'] = 'Muito Caro'
elif valor == 3:
    card['valor'] = 'Caro'
elif valor == 2:
    card['valor'] = 'Moderado'
elif valor == 1:
    card['valor'] = 'Barato'
else:
    card['valor'] = 0

In [30]:
card

{'nome': 'Figueira Rubayat',
 'local': 'São Paulo',
 'comida': ['Brasileira', 'Argentina', 'Grelhada', 'Churrascarias'],
 'valor': 'Muito Caro'}

**Obtendo o <font color=red>AVALIAÇÃO</font> do estabelecimento:**

Não consegui obter ainda a avaliação do estabelecimento.

**Criando DataFrame do Primeiro CARD**

In [31]:
card

{'nome': 'Figueira Rubayat',
 'local': 'São Paulo',
 'comida': ['Brasileira', 'Argentina', 'Grelhada', 'Churrascarias'],
 'valor': 'Muito Caro'}

In [32]:
dataset = pd.DataFrame.from_dict(card, orient = 'index').T
dataset

,nome,local,comida,valor
0,Figueira Rubayat,São Paulo,"[Brasileira, Argentina, Grelhada, Churrascarias]",Muito Caro


### <font color=#3498DB>Obtendo os dados de todos os CARDs da primeira página

In [33]:
anuncios = soup.find('div', {'class': 'restaurant_container'}).findAll('div', {'class': 'restaurant_row show'})

# Coletando informações dos CARDS
cards = []
for anuncio in anuncios:
    card = {}
    # Nome
    card['nome'] = anuncio.find('div', {'class': 'title'}).get_text().split(' /')[0]

    # Local
    local = anuncio.find('div',{'class':'number'})
    if local == None:
        continue
    else:
        local = local.get_text()
        local = local.split()
        local = local[7:]
        local = [' '.join(local)]
        local = str(local).strip("'[]'")
    card['local'] = local

    # Tipo de Comida
    comida = anuncio.find('div', {'class': 'info cuisine'}).get_text()
    comida = comida.split(', ')
    card['comida'] = comida
    
    # Valor
    valor = anuncio.find('span', {'class': 'cost'}).findAll('i')
    valor = len(valor)
    if valor == 4:
        card['valor'] = 'Muito Caro'
    elif valor == 3:
        card['valor'] = 'Caro'
    elif valor == 2:
        card['valor'] = 'Moderado'
    elif valor == 1:
        card['valor'] = 'Barato'
    else:
        card['valor'] = 0
    
    # Adicionando resultado a lista cards
    cards.append(card)

dataset_p1 = pd.DataFrame(cards)

In [34]:
dataset_p1

,nome,local,comida,valor
0,Figueira Rubayat,São Paulo,"[Brasileira, Argentina, Grelhada, Churrascarias]",Muito Caro
1,CT Boucherie,Rio de Janeiro,"[Francesa, Churrascarias, Opções vegetarianas]",Muito Caro
2,Tordesilhas,São Paulo,"[Brasileira, Opções vegetarianas]",Caro
3,Oro,Rio de Janeiro,"[Brasileira, Opções vegetarianas]",Muito Caro
4,D.O.M.,São Paulo,"[Brasileira, Opções vegetarianas]",Muito Caro
5,l'atelier du cuisinier,Rio de Janeiro,"[Francesa, Opções vegetarianas]",Caro
6,Fogo de Chão Jardins,São Paulo,"[Brasileira, Churrascarias, Grelhada, Opções v...",Muito Caro
7,L'Etoile Restaurante,Rio de Janeiro,"[Francesa, Europeia, Internacional, Opções veg...",Muito Caro
8,Restaurante Mestiço,São Paulo,"[Tailandesa, Opções vegetarianas]",Caro
9,Rio Scenarium,Rio de Janeiro,"[Brasileira, Latino-americana, Sul Americana]",Caro


### <font color=#3498DB>Obtendo uma quantidade considerável de cards

In [35]:
import time
cards = []
for i in range(1, 10):
    url = 'https://restaurantguru.com.br/restaurant-Brazil-t1'+'/'+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')
    
    anuncios = soup.find('div', {'class': 'restaurant_container'}).findAll('div', {'class': 'restaurant_row show'})
    
    for anuncio in anuncios:
        card = {}

        # Nome
        card['nome'] = anuncio.find('div', {'class': 'title'}).get_text().split(' /')[0]

        # Local
        local = anuncio.find('div', {'class':'number'})
        if local == None:
            continue
        else:
            local = local.get_text()
            local = local.split()
            local = local[7:]
            local = [' '.join(local)]
            local = str(local).strip("'[]'")
        card['local'] = local

        # Tipo de Comida
        comida = anuncio.find('div',{'class':'info cuisine'})
        if comida == None:
            continue
        else:
            comida = comida.get_text()
            comida = comida.split(', ')
            card['comida'] = comida

        # Valor
        valor = anuncio.find('span', {'class': 'cost'}).findAll('i')
        valor = len(valor)
        if valor == 4:
            card['valor'] = 'Muito Caro'
        elif valor == 3:
            card['valor'] = 'Caro'
        elif valor == 2:
            card['valor'] = 'Moderado'
        elif valor == 1:
            card['valor'] = 'Barato'
        else:
            card['valor'] = 0

        # Adicionando resultado a lista cards
        cards.append(card)
    time.sleep(3)
dataset = pd.DataFrame(cards)

### <font color=#3498DB>Exportando tabela para um arquivo csv

In [36]:
dataset.to_csv('restaurantes.csv', index = False)

In [37]:
data = pd.read_csv('restaurantes.csv')

In [39]:
data

,nome,local,comida,valor
0,Figueira Rubayat,São Paulo,"['Brasileira', 'Argentina', 'Grelhada', 'Churr...",Muito Caro
1,CT Boucherie,Rio de Janeiro,"['Francesa', 'Churrascarias', 'Opções vegetari...",Muito Caro
2,Tordesilhas,São Paulo,"['Brasileira', 'Opções vegetarianas']",Caro
3,Oro,Rio de Janeiro,"['Brasileira', 'Opções vegetarianas']",Muito Caro
4,D.O.M.,São Paulo,"['Brasileira', 'Opções vegetarianas']",Muito Caro
...,...,...,...,...
163,Balcone Restô,Fortaleza,"['Pizza', 'Italiana', 'Frutos do mar', 'Opções...",Moderado
164,Casa Vilharquide,Paraguaçu Paulista,"['Brasileira', 'Europeia', 'Alemã', 'Portugues...",Moderado
165,Basílico Cozinha Artesanal,Amparo,['Italiana'],Moderado
166,Dopo Lavoro Cantina Italiana,São Paulo,['Italiana'],Moderado
